In [1]:
import pickle

import pandas as pd
import scipy.sparse
import scipy.sparse
from sklearn import preprocessing

from lightfm import LightFM

In [2]:
df = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/amazon-clothes/5-core/train.parquet')
test_df = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/amazon-clothes/5-core/test.parquet')
categories = pd.read_json('/pio/scratch/1/recommender_systems/interim/Amazon/meta_Clothing_Shoes_and_Jewelry_categories.json', lines=True)

In [3]:
interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/5_core_interactions.npz')
test_interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/5_core_test_interactions.npz')
dataset = pd.read_pickle('/pio/scratch/1/i313924/data/lightfm_data/5_core_dataset.pkl')

In [4]:
items = df['asin'].drop_duplicates()
item_mapping = dataset.mapping()[2]
items_categories = categories[categories.asin.isin(items)].drop_duplicates()
without_category = set(items.values) - set(items_categories.asin)

In [5]:
le = preprocessing.LabelEncoder()
unique_categories = items_categories.category_1.unique()
le.fit(unique_categories)

LabelEncoder()

In [6]:
items_categories.category_1 = le.transform(items_categories.category_1)
for item in without_category:
    items_categories.loc[item_mapping[item]] = pd.Series({'category_1': -1, 'asin': item})
items_categories.index = items_categories.asin.map(lambda x: item_mapping[x])
items_categories = items_categories.sort_index()

In [9]:
item_category_mapping = items_categories.category_1

In [10]:
item_category_mapping

asin
0         6
1         5
2         5
3         5
4         5
         ..
242395    4
242396    5
242397    5
242398    5
242399    5
Name: category_1, Length: 242400, dtype: int64

In [11]:
model = LightFM(no_components=100, loss='warp', learning_schedule='adadelta', num_categories=len(unique_categories), item_category_mapping=item_category_mapping)

my new implementation


In [12]:
num_epochs = [5, 25, 50, 250, 500, 1000]
remaining = [num_epochs[0]] + [num_epochs[i] - num_epochs[i-1] for i in range(1, len(num_epochs))]
for i, epochs in enumerate(num_epochs):
    model.fit_partial(interactions, verbose=True, epochs=remaining[i], num_threads=8)
    pickle.dump(model, open(f'/pio/scratch/1/i313924/data/lightfm_data/model_{epochs}_epochs_all_clusters.pkl', 'wb'), protocol=4)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

category a:  9 category_b:  5


Epoch:  20%|██        | 1/5 [00:10<00:41, 10.33s/it]

category a:  2 category_b:  7


Epoch:  40%|████      | 2/5 [00:20<00:30, 10.30s/it]

...

Epoch: 100%|█████████▉| 498/500 [49:09<00:10,  5.37s/it]

category a:  2 category_b:  3


Epoch: 100%|█████████▉| 499/500 [49:13<00:05,  5.22s/it]

category a:  2 category_b:  4


Epoch: 100%|██████████| 500/500 [49:18<00:00,  5.92s/it]
